In [ ]:
!pip install fancyimpute

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ---------------- Imports ----------------
# Data manipulation and analysis
import pandas as pd
import numpy as np
import xgboost as xgb

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Machine learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Machine learning - Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from fancyimpute import KNN

# Machine learning - Metrics
from sklearn.metrics import mean_squared_error

# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Upload csv
df = pd.read_csv('/content/drive/MyDrive/פרויקט למידת מכונה/test.csv')

In [ ]:
df.shape

(7662, 291)

In [ ]:
# Identifying missing values in each feature
missing_values = df.isnull().sum()

# Calculating the percentage of missing values for each feature
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Combining both the count and percentage of missing values into a DataFrame
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Sorting the features by percentage of missing values in descending order
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)

missing_df

,Missing Values,Percentage
hospital_beds_raion,3418,44.609762
cafe_sum_500_max_price_avg,3159,41.229444
cafe_avg_price_500,3159,41.229444
cafe_sum_500_min_price_avg,3159,41.229444
preschool_quota,1596,20.830070
school_quota,1595,20.817019
cafe_avg_price_1000,1222,15.948838
cafe_sum_1000_max_price_avg,1222,15.948838
cafe_sum_1000_min_price_avg,1222,15.948838
build_count_mix,1218,15.896633


In [ ]:
# Split cols to categorical and numerical
# Modified to include both string and numeric columns
columns_of_interest = df.select_dtypes(include=['object', 'int64', 'float64']).columns

categorial_features = ['material', 'state', 'product_type', 'school_education_centers_top_20_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'culture_objects_top_25', 'culture_objects_top_25_raion', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'build_count_foam', 'build_count_mix', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ID_railroad_terminal', 'ecology', 'trc_count_500', 'cafe_count_500_price_high', 'mosque_count_500', 'market_count_500', 'cafe_count_1000_price_high', 'mosque_count_1000', 'market_count_1000', 'mosque_count_1500', 'market_count_1500', 'mosque_count_2000', 'market_count_2000', 'mosque_count_3000', 'mosque_count_5000']

print("categorial_features", categorial_features)

all_numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_features = [feature for feature in all_numeric_features if feature not in categorial_features]

# Exclude categorical features that are numerically encoded from the numeric_features list (not to be in categorial numeric)
numeric_features = [feature for feature in all_numeric_features if feature not in categorial_features]

print("Final numeric features, excluding categorical ones:", numeric_features)

categorial_features ['material', 'state', 'product_type', 'school_education_centers_top_20_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'culture_objects_top_25', 'culture_objects_top_25_raion', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'build_count_foam', 'build_count_mix', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ID_railroad_terminal', 'ecology', 'trc_count_500', 'cafe_count_500_price_high', 'mosque_count_500', 'market_count_500', 'cafe_count_1000_price_high', 'mosque_count_1000', 'market_count_1000', 'mosque_count_1500', 'market_count_1500', 'mosque_count_2000', 'market_count_2000', 'mosque_count_3000', 'mosque_count_5000']
Final numeric features, excluding categorical ones: ['id', 'full_sq', 'life_sq', 'floor', 'max_floor', 'build_year', 'num_room', 'kitch_sq', 'area_m', 'raion_popul', 'green_zone_part', 

In [ ]:
# Select columns that are integers
int_features = df.select_dtypes(include=['int64']).columns.tolist()

# Select columns that are floats
float_features = df.select_dtypes(include=['float64']).columns.tolist()

# Print the lists
print("Integer features:", int_features)
print("Float features:", float_features)

Integer features: ['id', 'floor', 'max_floor', 'material', 'num_room', 'raion_popul', 'children_preschool', 'preschool_education_centers_raion', 'children_school', 'school_education_centers_raion', 'school_education_centers_top_20_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', 'culture_objects_top_25_raion', 'shopping_centers_raion', 'office_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'ID_metro', 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2', 'ID_railroad_terminal', 'ID_bus_terminal', 'office_count_500', 'office_sqm_500', 'trc_count_500', 'trc_sqm_500', 'cafe_count_500', 'cafe_count_500_

In [ ]:
# Print initial data types for verification
print("Initial data types (selected):")
print(df[float_features].dtypes)

for col in float_features:
    # Check if all non-NaN entries in the column are integers
    if df[col].dropna().apply(lambda x: x % 1 == 0).all():
        print(f"Converting {col} to integers.")

        # Convert to Int64 dtype while preserving NaNs
        df[col] = df[col].astype('Int64')

# Verify changes by checking data types again
print("\nUpdated data types (selected):")
conversion_results = df[float_features].dtypes
print(conversion_results)

Initial data types (selected):
full_sq                        float64
life_sq                        float64
build_year                     float64
kitch_sq                       float64
state                          float64
                                ...   
green_part_5000                float64
prom_part_5000                 float64
cafe_sum_5000_min_price_avg    float64
cafe_sum_5000_max_price_avg    float64
cafe_avg_price_5000            float64
Length: 116, dtype: object
Converting build_year to integers.
Converting state to integers.
Converting preschool_quota to integers.
Converting school_quota to integers.
Converting hospital_beds_raion to integers.
Converting raion_build_count_with_material_info to integers.
Converting build_count_block to integers.
Converting build_count_wood to integers.
Converting build_count_frame to integers.
Converting build_count_brick to integers.
Converting build_count_monolith to integers.
Converting build_count_panel to integers.
Converting b

In [ ]:
# Select columns that are integers
int_features = df.select_dtypes(include=['int64']).columns.tolist()

# Select columns that are floats
float_features = df.select_dtypes(include=['float64']).columns.tolist()

# Print the lists
print("Integer features:", int_features)
print("Float features:", float_features)

Integer features: ['id', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'state', 'raion_popul', 'children_preschool', 'preschool_quota', 'preschool_education_centers_raion', 'children_school', 'school_quota', 'school_education_centers_raion', 'school_education_centers_top_20_raion', 'hospital_beds_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', 'culture_objects_top_25_raion', 'shopping_centers_raion', 'office_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_mo

In [ ]:
# Impute missing values for integers with median
df[int_features] = df[int_features].fillna(df[int_features].median())

# Impute missing values for floats with mean
df[float_features] = df[float_features].fillna(df[float_features].mean())

Model Enhancement - Feature Engineering

In [ ]:
# Polynomial Features
df['full_sq ** 2'] = df['full_sq'] ** 2
df['life_sq ** 2'] = df['life_sq'] ** 2
df['num_room ** 2'] = df['num_room'] ** 2

<ipython-input-96-0ff7a6539eb2>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['full_sq ** 2'] = df['full_sq'] ** 2
<ipython-input-96-0ff7a6539eb2>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['life_sq ** 2'] = df['life_sq'] ** 2
<ipython-input-96-0ff7a6539eb2>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

In [ ]:
# Interaction Terms
# Transport-related Interactions
df['railroad_station_avto_km_x_railroad_station_avto_min'] = df['railroad_station_avto_km'] * df['railroad_station_avto_min']

# Distance-related Interactions
df['mkad_km_x_ttk_km'] = df['mkad_km'] * df['ttk_km']
df['kremlin_km_x_big_road1_km'] = df['kremlin_km'] * df['big_road1_km']

# Healthcare-related Interactions
df['public_healthcare_km_x_hospice_morgue_km'] = df['public_healthcare_km'] * df['hospice_morgue_km']
df['university_km_x_public_healthcare_km'] = df['university_km'] * df['public_healthcare_km']

# Cultural-related Interactions
df['theater_km_x_museum_km'] = df['theater_km'] * df['museum_km']
df['church_synagogue_km_x_mosque_km'] = df['church_synagogue_km'] * df['mosque_km']

# Commercial-related Interactions
df['shopping_centers_km_x_office_km'] = df['shopping_centers_km'] * df['office_km']
df['market_shop_km_x_office_count_500'] = df['market_shop_km'] * df['office_count_500']

# Recreational-related Interactions
df['fitness_km_x_swim_pool_km'] = df['fitness_km'] * df['swim_pool_km']
df['ice_rink_km_x_stadium_km'] = df['ice_rink_km'] * df['stadium_km']

# Feature Engineering
df['living_to_total_area_ratio'] = df['life_sq'] / df['full_sq']
df['area_per_room'] = df['full_sq'] / df['num_room']

df['timestamp'] = pd.to_datetime(df['timestamp'])

df['transaction_year'] = df['timestamp'].dt.year
df['rooms_per_sqm'] = df['num_room'] / df['full_sq']
df['age_of_building'] = df['transaction_year'] - df['build_year']

<ipython-input-97-fe01ade359e2>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['railroad_station_avto_km_x_railroad_station_avto_min'] = df['railroad_station_avto_km'] * df['railroad_station_avto_min']
<ipython-input-97-fe01ade359e2>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['mkad_km_x_ttk_km'] = df['mkad_km'] * df['ttk_km']
<ipython-input-97-fe01ade359e2>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [ ]:
# List of interaction terms
interaction_columns = [
    'railroad_station_avto_km_x_railroad_station_avto_min',
    'mkad_km_x_ttk_km',
    'kremlin_km_x_big_road1_km',
    'public_healthcare_km_x_hospice_morgue_km',
    'university_km_x_public_healthcare_km',
    'theater_km_x_museum_km',
    'church_synagogue_km_x_mosque_km',
    'shopping_centers_km_x_office_km',
    'market_shop_km_x_office_count_500',
    'fitness_km_x_swim_pool_km',
    'ice_rink_km_x_stadium_km'
]

# Selecting the interaction terms
df_interactions = df[interaction_columns]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the interaction terms
df_interactions_scaled = scaler.fit_transform(df_interactions)

# Replace the original interaction columns in the DataFrame
df[interaction_columns] = df_interactions_scaled

# Now df contains the scaled version of your interaction terms within the range [0, 1]
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,church_synagogue_km_x_mosque_km,shopping_centers_km_x_office_km,market_shop_km_x_office_count_500,fitness_km_x_swim_pool_km,ice_rink_km_x_stadium_km,living_to_total_area_ratio,area_per_room,transaction_year,rooms_per_sqm,age_of_building
0,30474,2015-07-01,39.0,20.700000,2,9,1,1998,1,8.9,...,0.007401,0.039122,0.0,0.000583,0.074467,0.530769,39.00,2015,0.025641,17
1,30475,2015-07-01,79.2,30.896577,8,17,1,0,3,1.0,...,0.064729,0.021862,0.0,0.003919,0.018423,0.390108,26.40,2015,0.037879,2015
2,30476,2015-07-01,40.5,25.100000,3,5,2,1960,2,4.8,...,0.053245,0.005867,0.0,0.000905,0.001383,0.619753,20.25,2015,0.049383,55
3,30477,2015-07-01,62.8,36.000000,17,17,1,2016,2,62.8,...,0.005987,0.175716,0.0,0.027021,0.087234,0.573248,31.40,2015,0.031847,-1
4,30478,2015-07-01,40.0,40.000000,17,17,1,0,1,1.0,...,0.068309,0.025796,0.0,0.002694,0.011641,1.000000,40.00,2015,0.025000,2015


In [ ]:
# timestamp to datetime
# df['timestamp'] = pd.to_datetime(df['timestamp'])
# add the year and month columns
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
# drop timestamp
df.drop('timestamp', axis=1, inplace=True)

<ipython-input-99-33c0d80992e8>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['year'] = df['timestamp'].dt.year
<ipython-input-99-33c0d80992e8>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['month'] = df['timestamp'].dt.month


In [ ]:
categorial_features.append('sub_area')
# Initializing the OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit and transform the categorical features
encoded_categorical = encoder.fit_transform(df[categorial_features])

# Create a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorial_features))

# Reset index to ensure concatenation works correctly if indices are not aligned
df = df.reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Drop the original categorical columns from X
df = df.drop(columns=categorial_features)

# Concatenate the encoded categorical features with the rest of the dataset (numeric features already in X)
df = pd.concat([df, encoded_df], axis=1)
# Now, X_final is ready with one-hot encoded categorical features and numeric features, ready for model training

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from sklearn.ensemble import RandomForestRegressor

# imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=5, random_state=0)
# imputed_data = imputer.fit_transform(df)
# df.loc[:, df] = imputed_data

In [ ]:
df.shape

(7662, 566)

Random forest MSE = 6274411116013.969
RandomForestClassifier MSE =

XGBoost MSE = 4947192476493.555
LightGBM MSE = 4877120243679.092 Best !!

In [ ]:
from google.colab import files
df.to_csv('Manipulated_Test.csv', index=False)
files.download('Manipulated_Test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>